In [ ]:
import os
gpu_num = "" # Use "" to use the CPU
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

resolution = [480,320] # increase for higher quality of renderings

# Allows to exit cell execution in Jupyter
class ExitCell(Exception):
    def _render_traceback_(self):
        pass

# Import Sionna
try:
    import sionna
except ImportError as e:
    # Install Sionna if package is not already installed
    import os
    os.system("pip install sionna")
    import sionna

# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')

tf.random.set_seed(1) # Set global random seed for reproducibility

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import time

# Import Sionna RT components
from sionna.rt import load_scene, Transmitter, Receiver, PlanarArray, RadioMaterial, LambertianPattern
from sionna.constants import PI

# For link-level simulations
from sionna.channel import cir_to_ofdm_channel, subcarrier_frequencies, OFDMChannel, ApplyOFDMChannel, CIRDataset
from sionna.nr import PUSCHConfig, PUSCHTransmitter, PUSCHReceiver
from sionna.utils import compute_ber, ebnodb2no, PlotBER
from sionna.ofdm import KBestDetector, LinearDetector
from sionna.mimo import StreamManagement

In [ ]:
# Load integrated scene
scene = load_scene("../../scenes/messe_obstacles/Messe.xml")

In [ ]:
scene.get("elm__4").radio_material = "itu_concrete"
scene.get("elm__5").radio_material = "itu_concrete"

In [ ]:
centers = {"big_hall":[0, 0, 1.5],"small_hall":[-100, 0, 1.5]}

In [ ]:
sampling_frequency = 10e5
sample_m = 62.0 / sampling_frequency

c0 = sionna.SPEED_OF_LIGHT

fc = 3.7e9
scene.frequency = fc # in Hz; implicitly updates RadioMaterials

scene.synthetic_array = False # If set to False, ray tracing will be done per antenna element (slower for large arrays)

In [ ]:
da = 0.1
M = 4
N = 4

scene._clear()

# Configure antenna array for all transmitters
scene.tx_array = PlanarArray(num_rows=N,
                             num_cols=M,
                             vertical_spacing=da,
                             horizontal_spacing=da,
                             pattern="dipole",
                             polarization="V")

# Configure antenna array for all receivers
scene.rx_array = PlanarArray(num_rows=1,
                             num_cols=1,
                             vertical_spacing=da,
                             horizontal_spacing=da,
                             pattern="tr38901",
                             polarization="V")
                             #polarization="cross")

In [ ]:
transmitters_dipole = [
    {
        "name":"big_hall",
        "position":[0,0,18],
        "orientation":[0.0, 0.0, 0.0],
        "antenna_type":"dipole"
    },
    {
        "name":"small_hall",
        "position":[-100,0,18],
        "orientation":[0.0, 0.0, 0.0],
        "antenna_type":"dipole"
    }
]

transmitters_half_dipole_front = [
    {
        "name":"big_hall",
        "position":[0,78,18],
        "orientation":[0.0, 0.0, 0.0],
        "antenna_type":"dipole"
    },
    {
        "name":"small_hall",
        "position":[-100,53,18],
        "orientation":[0.0, 0.0, 0.0],
        "antenna_type":"dipole"
    }
]

transmitters_tr_front = [
    {
        "name":"big_hall",
        "position":[0,78,18],
        "orientation":tf.Variable([0.0, 0.0, 0.0], dtype=tf.float32),
        "antenna_type":"dipole"
    },
    {
        "name":"small_hall",
        "position":[-100,53,18],
        "orientation":tf.Variable([0.0, 0.0, 0.0], dtype=tf.float32),
        "antenna_type":"dipole"
    }
]

transmitters_tr_sectors = [
    {
        "name":"big_hall_1",
        "position":[0,0,18],
        "orientation":tf.Variable([0.0, 0.0, 0.0], dtype=tf.float32),
        "antenna_type":"dipole"
    },
    {
        "name":"big_hall_2",
        "position":[0,0,18],
        "orientation":tf.Variable([0.0, 0.0, 0.0], dtype=tf.float32),
        "antenna_type":"dipole"
    },
    {
        "name":"big_hall_3",
        "position":[0,0,18],
        "orientation":tf.Variable([0.0, 0.0, 0.0], dtype=tf.float32),
        "antenna_type":"dipole"
    },
    {
        "name":"small_hall_1",
        "position":[-100,0,18],
        "orientation":tf.Variable([0.0, 0.0, 0.0], dtype=tf.float32),
        "antenna_type":"dipole"
    },
    {
        "name":"small_hall_2",
        "position":[-100,0,18],
        "orientation":tf.Variable([0.0, 0.0, 0.0], dtype=tf.float32),
        "antenna_type":"dipole"
    },
    {
        "name":"small_hall_3",
        "position":[-100,0,18],
        "orientation":tf.Variable([0.0, 0.0, 0.0], dtype=tf.float32),
        "antenna_type":"dipole"
    }
]

In [ ]:
for i in range(len(transmitters_half_dipole_front)):
    tx = Transmitter(name=transmitters_half_dipole_front[i]["name"],
                 position=transmitters_half_dipole_front[i]["position"],
                 orientation=transmitters_half_dipole_front[i]["orientation"])

    scene.add(tx)
    print(transmitters_half_dipole_front[i]["name"])


In [ ]:
import numpy as np
import tensorflow as tf

# Configure an SGD optimizer
optimizer = tf.keras.optimizers.legacy.RMSprop(0.1)

# Number of training steps
num_steps = 11

# Weighting factors for the loss terms
weight_sinr = 1
weight_coverage = 1

def train_step():
    """A single training step"""
    with tf.GradientTape() as tape:
        print(tape.watched_variables())
        
        # Compute coverage of the target area
        target_cm = scene.coverage_map(num_samples=1e5, diffraction=False, scattering=False,
                            cm_center=[-50, 0, 1.5],
                            cm_orientation=[0,0,0],
                            cm_size=[400,200],
                            cm_cell_size=[1,1])
        
        # Convert to tf.float32 to ensure dtype consistency
        target_cm_tensor = target_cm.as_tensor()
        
        # Compute coverage rate
        scaling = 1e12
        rate = tf.reduce_mean(tf.math.log(1. + target_cm_tensor * scaling))
        coverage_loss = -rate  # Note the negative sign to make it a loss
        # Combine SINR loss and coverage loss
        loss = weight_coverage * coverage_loss
    # Compute gradients and apply through the optimizer
    grads = tape.gradient(loss, tape.watched_variables())
    optimizer.apply_gradients(zip(grads, tape.watched_variables()))
    return loss

for step in range(num_steps):
    loss = train_step()
    print(f"Training step {step} - Loss: {loss.numpy():.2E} - tx orientation: {scene.transmitters['big_hall'].orientation.numpy()}", end='\r')
    


In [ ]:
cm_new = scene.coverage_map(cm_center=[-50, 0, 1.5],
                            cm_orientation=[0,0,0],
                            cm_size=[400,200],
                            cm_cell_size=[0.5,0.5])

In [ ]:
cm_new.show(tx=0);
cm_new.show(tx=1);

In [ ]:
scene.preview(coverage_map=cm_new, cm_tx=0, clip_at=18)

In [ ]:
import numpy as np

def calculate_sinr_matrix(signal_matrices, noise_power):
    # Convert the noise power to linear scale
    noise_linear = 10 ** (noise_power / 10)

    # Number of matrices (4 antennas) and dimensions of the matrix
    num_antennas = len(signal_matrices)
    rows, cols = signal_matrices[0].shape

    # Initialize the SINR matrix with zeros
    sinr_matrix = np.zeros((rows, cols))

    # Optionally store the index of the reference signal (strongest signal) for each point
    reference_signal_indices = np.zeros((rows, cols), dtype=int)

    # Loop through each point in the matrices
    for i in range(rows):
        for j in range(cols):
            # Extract the signal power values at this point from all antennas
            signal_values = [signal_matrices[k][i, j] for k in range(num_antennas)]

            # Convert signal values from dB to linear scale
            signal_values_linear = [10 ** (sv / 10) for sv in signal_values]

            # Find the reference signal (the strongest signal)
            reference_signal_linear = max(signal_values_linear)
            reference_signal_index = signal_values_linear.index(reference_signal_linear)

            # Store the index of the reference signal (optional)
            reference_signal_indices[i, j] = reference_signal_index

            # Calculate interference (sum of all signals except the reference signal)
            interference_linear = sum(signal_values_linear) - reference_signal_linear

            # Calculate SINR at this point
            sinr_linear = reference_signal_linear / (interference_linear+noise_linear)

            sinr_val = 10 * np.log10(sinr_linear)
            if sinr_val > 30:
                sinr_matrix[i, j] = 30
            elif sinr_val < 0: sinr_matrix[i, j] = -np.inf
            else: sinr_matrix[i, j] = 10 * np.log10(sinr_linear)


    return sinr_matrix, reference_signal_indices

sinr_matrix, _= calculate_sinr_matrix([cm_0,cm_1,cm_2,cm_3,cm_4,cm_5,cm_6], -999999999999999999)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap, BoundaryNorm
colors = [(0.9, 0, 0), (1, 1, 0), (0.3, 0.8, 0)]  # R -> G -> B
n_bins = [100]  # Discretizes the interpolation into bins
cmap_name = 'my_list'
fig, ax = plt.subplots(1, 1)
cm = LinearSegmentedColormap.from_list(
    cmap_name, colors, N=100)
# Remove ticks from the x and y axes
ax.set_xticks([])
ax.set_yticks([])
# Fewer bins will result in "coarser" colomap interpolation
im = ax.imshow(sinr_matrix, interpolation='nearest', origin='lower', cmap=cm, vmin=-5, vmax=30)
fig.colorbar(im, ax=ax, orientation='horizontal')
# fig.axes[0].scatter(*cm_new._tx_pos[0], marker='.', c='r')
# fig.axes[0].scatter(*cm_new._tx_pos[1], marker='.', c='r')
# fig.axes[0].scatter(*cm_new._tx_pos[2], marker='.', c='r')
# fig.axes[0].scatter(*cm_new._tx_pos[3], marker='.', c='r')
# fig.axes[0].scatter(*cm_new._tx_pos[4], marker='.', c='r')
# fig.axes[0].scatter(*cm_new._tx_pos[5], marker='.', c='r')
# fig.axes[0].scatter(*cm_new._tx_pos[6], marker='.', c='r')
plt.savefig("sinr.png", dpi=300)

In [ ]:
import numpy as np

# Define the threshold
T = 13

# Create binary matrix B
B = (sinr_matrix > T).astype(np.float32)

sns.heatmap(B)
plt.show()

In [ ]:
counter = 0
summ = []
for i,row in enumerate(sinr_matrix):
    if i < 300 and i > 180:
        for j,item in enumerate(row):
            if j < 580 and j > 140:
                if item > 0.5:
                    summ.append(item)
                if item > 0.5 and item < 5: print(item,i,j)
            
            

print(min(summ),max(summ),np.mean(summ))

In [ ]:
print(sinr_matrix.min(), sinr_matrix.max())

In [ ]:
from matplotlib.colors import LinearSegmentedColormap, BoundaryNorm
colors = [(0.9, 0, 0), (1, 1, 0), (0.3, 0.8, 0)]  # R -> G -> B
n_bins = [100]  # Discretizes the interpolation into bins
cmap_name = 'my_list'
fig, ax = plt.subplots(1, 1)
cm = LinearSegmentedColormap.from_list(
    cmap_name, colors, N=100)
# Remove ticks from the x and y axes
ax.set_xticks([])
ax.set_yticks([])
# Fewer bins will result in "coarser" colomap interpolation
im = ax.imshow(sinr_matrix, interpolation='nearest', origin='lower', cmap=cm)
fig.colorbar(im, ax=ax, orientation='horizontal')

In [ ]:
th_norm

In [ ]:
cdict

In [ ]:
def NonLinCdict(steps, hexcol_array):
    cdict = {'red': (), 'green': (), 'blue': ()}
    for s, hexcol in zip(steps, hexcol_array):
        rgb =matplotlib.colors.hex2color(hexcol)
        cdict['red'] = cdict['red'] + ((s, rgb[0], rgb[0]),)
        cdict['green'] = cdict['green'] + ((s, rgb[1], rgb[1]),)
        cdict['blue'] = cdict['blue'] + ((s, rgb[2], rgb[2]),)
    return cdict

hc = ['#e5e5ff', '#acacdf', '#7272bf', '#39399f', '#000080']
th = [0, 0.1, 0.5, 0.9, 1]

cdict = NonLinCdict(th, hc)
cdict

In [ ]:
from sionna.rt.utils import r_hat, rotation_matrix, theta_phi_from_unit_vec, expand_to_rank
import tensorflow as tf
import numpy as np

In [ ]:
# Convert the list to a NumPy array
angles = np.array([-2.80684, 0.36026, 0.027153])

# Now, pass the NumPy array to the rotation_matrix function
rot_mat = tf.transpose(rotation_matrix(angles))

In [ ]:
np.arctan2(rot_mat[1][1],rot_mat[0][1]) * 180 / 3.14

In [ ]:
np.arcsin(-rot_mat[2][1]) * 180 / 3.14